In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from utils.model_val_metrics_helper import select_threshold, metric_null, plot_intervention, MEWS
from tableone import TableOne

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.options.display.float_format = '{:,.4f}'.format

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/matplotlib_inline/config.py:75: DeprecationWarning: InlineBackend._figure_format_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_format_changed(self, name, old, new):


# I. Load and Preprocess Data

In [2]:
pred_df = pd.read_csv('data/main_prospective_val_df_20220805_20220930_20221010.csv')
pred_df_full = pd.read_pickle('data/pred_df_full_20220805_20220930_20221010.pkl')
pred_df_full.iloc[:, 4:]

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (228) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,HR,Resp,Temp,Weight,BMI,SpO2,gcs_total_score,MAP,ALKALINE PHOSPHATASE,APTT,"ARTERIAL PCO2, POC","ARTERIAL PH, POC","ARTERIAL PO2, POC",AST,BILIRUBIN TOTAL,BLOOD UREA NITROGEN,CALCIUM,CHLORIDE,CREATININE,FIBRINOGEN,"FIO2, ABG",GLUCOSE,HEMATOCRIT,HEMOGLOBIN,"LACTATE, ABG",MAGNESIUM,"OXYGEN PERCENT, ABG","PCO2, ABG","PH, ABG","PHOSPHORUS, INORGANIC",PLATELET COUNT,"PO2, ABG",POTASSIUM,SODIUM,WBC,AgeAtAdmissioninYears,sex_Female,sex_Male,etco2_interval_f1,etco2_interval_f2,etco2_diff,SBP_interval_f1,SBP_interval_f2,SBP_diff,DBP_interval_f1,DBP_interval_f2,DBP_diff,HR_interval_f1,HR_interval_f2,HR_diff,Resp_interval_f1,Resp_interval_f2,Resp_diff,Temp_interval_f1,Temp_interval_f2,Temp_diff,Weight_interval_f1,Weight_interval_f2,Weight_diff,BMI_interval_f1,BMI_interval_f2,BMI_diff,SpO2_interval_f1,SpO2_interval_f2,SpO2_diff,gcs_total_score_interval_f1,gcs_total_score_interval_f2,gcs_total_score_diff,MAP_interval_f1,MAP_interval_f2,MAP_diff,ALKALINE PHOSPHATASE_interval_f1,ALKALINE PHOSPHATASE_interval_f2,ALKALINE PHOSPHATASE_diff,APTT_interval_f1,APTT_interval_f2,APTT_diff,"ARTERIAL PCO2, POC_interval_f1","ARTERIAL PCO2, POC_interval_f2","ARTERIAL PCO2, POC_diff","ARTERIAL PH, POC_interval_f1","ARTERIAL PH, POC_interval_f2","ARTERIAL PH, POC_diff","ARTERIAL PO2, POC_interval_f1","ARTERIAL PO2, POC_interval_f2","ARTERIAL PO2, POC_diff",AST_interval_f1,AST_interval_f2,AST_diff,BILIRUBIN TOTAL_interval_f1,BILIRUBIN TOTAL_interval_f2,BILIRUBIN TOTAL_diff,BLOOD UREA NITROGEN_interval_f1,BLOOD UREA NITROGEN_interval_f2,BLOOD UREA NITROGEN_diff,CALCIUM_interval_f1,CALCIUM_interval_f2,CALCIUM_diff,CHLORIDE_interval_f1,CHLORIDE_interval_f2,CHLORIDE_diff,CREATININE_interval_f1,CREATININE_interval_f2,CREATININE_diff,"FIO2, ABG_interval_f1","FIO2, ABG_interval_f2","FIO2, ABG_diff",GLUCOSE_interval_f1,GLUCOSE_interval_f2,GLUCOSE_diff,HEMATOCRIT_interval_f1,HEMATOCRIT_interval_f2,HEMATOCRIT_diff,HEMOGLOBIN_interval_f1,HEMOGLOBIN_interval_f2,HEMOGLOBIN_diff,"LACTATE, ABG_interval_f1","LACTATE, ABG_interval_f2","LACTATE, ABG_diff",MAGNESIUM_interval_f1,MAGNESIUM_interval_f2,MAGNESIUM_diff,"OXYGEN PERCENT, ABG_interval_f1","OXYGEN PERCENT, ABG_interval_f2","OXYGEN PERCENT, ABG_diff","PCO2, ABG_interval_f1","PCO2, ABG_interval_f2","PCO2, ABG_diff","PH, ABG_interval_f1","PH, ABG_interval_f2","PH, ABG_diff","PHOSPHORUS, INORGANIC_interval_f1","PHOSPHORUS, INORGANIC_interval_f2","PHOSPHORUS, INORGANIC_diff",PLATELET COUNT_interval_f1,PLATELET COUNT_interval_f2,PLATELET COUNT_diff,"PO2, ABG_interval_f1","PO2, ABG_interval_f2","PO2, ABG_diff",POTASSIUM_interval_f1,POTASSIUM_interval_f2,POTASSIUM_diff,SODIUM_interval_f1,SODIUM_interval_f2,SODIUM_diff,WBC_interval_f1,WBC_interval_f2,WBC_diff,HR_mean,O2Sat_mean,SBP_mean,MAP_mean,Resp_mean,O2Sat_median,SBP_median,MAP_median,Resp_median,O2Sat_min,SBP_min,MAP_min,Resp_min,O2Sat_max,SBP_max,MAP_max,Resp_max,O2Sat_std,SBP_std,MAP_std,Resp_std,O2Sat_dstd,SBP_dstd,MAP_dstd,Resp_dstd,HR_score,Temp_score,Resp_score,MAP_score,qsofa,SIRS,PredictedProbability,SepsisLabel_0.005,SepsisLabel_0.01,SepsisLabel_0.02,SepsisLabel_0.03,SepsisLabel_0.04,SepsisLabel_0.05,SepsisLabel_0.06,SepsisLabel_0.07,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time,Sepsis3
0,TH 6 MAIN PERI-OP,Tisch,0.0000,0.5000,2022-08-03 06:36:00,2022-08-03 06:06:00,20220803,NaN,132.0000,76.0000,88.0000,15.0000,36.1140,"3,086.4400",33.1100,97.0000,NaN,94.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.1657,0.0000,1.0000,0.0000,-1.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,1.0000,0.0000,NaN,0.0000,-1.0000,NaN,1.0000,0.0000,NaN,0.0000,-1.0000,NaN,0.0000,-1.0000,NaN,0.0000,-1.0000,NaN,0.0000,-1.0000,NaN,0

In [3]:
def calculate_sirs(pred_df_full):
    pred_df_sirs = pred_df_full[pred_df_full['SIRS'] == 1]
    pred_df_sirs.sort_values(by=['rel_time'], inplace=True)
    pred_df_sirs.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    pred_df_sirs['time_to_first_SIRS_alert'] = (pd.to_datetime(pred_df_sirs['abs_time']) - pd.to_datetime(pred_df_sirs['AdmissionInstant'])).dt.total_seconds() / 3600.
    pred_df_sirs['SIRS_Positive'] = 1
    return pred_df_sirs[['ID', 'SIRS_Positive', 'time_to_first_SIRS_alert']]
    
def calculate_qsofa(pred_df_full):
    pred_df_qsofa = pred_df_full[pred_df_full['qsofa'] == 1]
    pred_df_qsofa.sort_values(by=['rel_time'], inplace=True)
    pred_df_qsofa.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    pred_df_qsofa['time_to_first_qsofa_alert'] = (pd.to_datetime(pred_df_qsofa['abs_time']) - pd.to_datetime(pred_df_qsofa['AdmissionInstant'])).dt.total_seconds() / 3600.
    pred_df_qsofa['qSOFA_Positive'] = 1
    return pred_df_qsofa[['ID', 'qSOFA_Positive', 'time_to_first_qsofa_alert']]

def calculate_sepsis3(pred_df_full):
    pred_df_sepsis3 = pred_df_full[pred_df_full['Sepsis3'] == 1]
    pred_df_sepsis3.sort_values(by=['rel_time'], inplace=True)
    pred_df_sepsis3.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    pred_df_sepsis3['time_to_first_sepsis3_alert'] = (pd.to_datetime(pred_df_sepsis3['abs_time']) - pd.to_datetime(pred_df_sepsis3['AdmissionInstant'])).dt.total_seconds() / 3600.
    return pred_df_sepsis3[['ID', 'Sepsis3', 'time_to_first_sepsis3_alert']]

def calculate_mews(pred_df_full):
    mews = MEWS(pred_df_full)
    pred_df_full = pred_df_full.merge(mews, on = ['ID', 'rel_time'], how = "left")
    pred_df_full["MEWS"] = pred_df_full["MEWS"] >= 5
    pred_df_full['MEWS'] = pred_df_full['MEWS'].astype(int)
    
    pred_df_mews = pred_df_full[pred_df_full['MEWS'] == 1]
    pred_df_mews.sort_values(by=['rel_time'], inplace=True)
    pred_df_mews.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    pred_df_mews['time_to_first_mews_alert'] = (pd.to_datetime(pred_df_mews['abs_time']) - pd.to_datetime(pred_df_mews['AdmissionInstant'])).dt.total_seconds() / 3600.
    pred_df_mews['MEWS_Positive'] = 1
    return pred_df_mews[['ID', 'MEWS_Positive', 'time_to_first_mews_alert']]

def get_baseline_labs_vitals(pred_df_full):
    include_labs = ['WBC', 'GLUCOSE', 'PO2, ABG', 'FIO2, ABG', 'CREATININE', 'APTT', 
                'PLATELET COUNT', 'BILIRUBIN TOTAL', 'LACTATE, ABG']

    include_vitals = ['SBP', 'MAP', 'Temp', 'HR', 'Resp']
    
    vitals_labs = ['ID'] + include_vitals + include_labs 
    
    pred_df_full.sort_values(by=['rel_time'],ascending=True, inplace=True)
    pred_df_full_baseline = pred_df_full[vitals_labs]
    pred_df_full_baseline.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    pred_df_full_baseline.columns = ['ID'] + [str(cols) + '_' for cols in pred_df_full_baseline.columns if cols != 'ID']
    return pred_df_full_baseline
    
def create_sepsis_cohort(pred_df, pred_df_full):
    
    pred_df['time_to_first_ML_alert'] = (pd.to_datetime(pred_df['abs_time']) - pd.to_datetime(pred_df['AdmissionInstant'])).dt.total_seconds() / 3600.
    
    sirs_df = calculate_sirs(pred_df_full)
    pred_df = pred_df.merge(sirs_df, on=['ID'], how='left')
    pred_df['SIRS_Positive'] = pred_df['SIRS_Positive'].fillna(0)
    
    qsofa_df = calculate_qsofa(pred_df_full)
    pred_df = pred_df.merge(qsofa_df, on=['ID'], how='left')
    pred_df['qSOFA_Positive'] = pred_df['qSOFA_Positive'].fillna(0)
    
    sepsis3_df = calculate_sepsis3(pred_df_full)
    pred_df = pred_df.merge(sepsis3_df, on=['ID'], how='left')
    pred_df['Sepsis3'] = pred_df['Sepsis3'].fillna(0)
    
    mews_df = calculate_mews(pred_df_full)
    pred_df = pred_df.merge(mews_df, on=['ID'], how='left')
    pred_df['MEWS_Positive'] = pred_df['MEWS_Positive'].fillna(0)
    
    pred_df['Sepsis_Grouper'] = pred_df['Sepsis_Grouper'].fillna('Others, N=0')
    pred_df['Sepsis_Grouper'] = pred_df['Sepsis_Grouper'].apply(lambda x: x.split(', N=')[0])
    #pred_df_full_baseline = get_baseline_labs_vitals(pred_df_full)
    #pred_df = pred_df.merge(pred_df_full_baseline, on=['ID'], how='left')
    
    pred_df = pred_df.reset_index()
    return pred_df
    

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
pred_df = create_sepsis_cohort(pred_df, pred_df_full)
pred_df.iloc[:,4:]

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-3-dad158c1e07b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,DepartmentServiceGrouper,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,HR,Resp,Temp,Weight,BMI,SpO2,gcs_total_score,MAP,ALKALINE PHOSPHATASE,APTT,"ARTERIAL PCO2, POC","ARTERIAL PH, POC","ARTERIAL PO2, POC",AST,BILIRUBIN TOTAL,BLOOD UREA NITROGEN,CALCIUM,CHLORIDE,CREATININE,FIBRINOGEN,"FIO2, ABG",GLUCOSE,HEMATOCRIT,HEMOGLOBIN,"LACTATE, ABG",MAGNESIUM,"OXYGEN PERCENT, ABG","PCO2, ABG","PH, ABG","PHOSPHORUS, INORGANIC",PLATELET COUNT,"PO2, ABG",POTASSIUM,SODIUM,WBC,AgeAtAdmissioninYears,sex_Female,sex_Male,etco2_interval_f1,etco2_interval_f2,etco2_diff,SBP_interval_f1,SBP_interval_f2,SBP_diff,DBP_interval_f1,DBP_interval_f2,DBP_diff,HR_interval_f1,HR_interval_f2,HR_diff,Resp_interval_f1,Resp_interval_f2,Resp_diff,Temp_interval_f1,Temp_interval_f2,Temp_diff,Weight_interval_f1,Weight_interval_f2,Weight_diff,BMI_interval_f1,BMI_interval_f2,BMI_diff,SpO2_interval_f1,SpO2_interval_f2,SpO2_diff,gcs_total_score_interval_f1,gcs_total_score_interval_f2,gcs_total_score_diff,MAP_interval_f1,MAP_interval_f2,MAP_diff,ALKALINE PHOSPHATASE_interval_f1,ALKALINE PHOSPHATASE_interval_f2,ALKALINE PHOSPHATASE_diff,APTT_interval_f1,APTT_interval_f2,APTT_diff,"ARTERIAL PCO2, POC_interval_f1","ARTERIAL PCO2, POC_interval_f2","ARTERIAL PCO2, POC_diff","ARTERIAL PH, POC_interval_f1","ARTERIAL PH, POC_interval_f2","ARTERIAL PH, POC_diff","ARTERIAL PO2, POC_interval_f1","ARTERIAL PO2, POC_interval_f2","ARTERIAL PO2, POC_diff",AST_interval_f1,AST_interval_f2,AST_diff,BILIRUBIN TOTAL_interval_f1,BILIRUBIN TOTAL_interval_f2,BILIRUBIN TOTAL_diff,BLOOD UREA NITROGEN_interval_f1,BLOOD UREA NITROGEN_interval_f2,BLOOD UREA NITROGEN_diff,CALCIUM_interval_f1,CALCIUM_interval_f2,CALCIUM_diff,CHLORIDE_interval_f1,CHLORIDE_interval_f2,CHLORIDE_diff,CREATININE_interval_f1,CREATININE_interval_f2,CREATININE_diff,"FIO2, ABG_interval_f1","FIO2, ABG_interval_f2","FIO2, ABG_diff",GLUCOSE_interval_f1,GLUCOSE_interval_f2,GLUCOSE_diff,HEMATOCRIT_interval_f1,HEMATOCRIT_interval_f2,HEMATOCRIT_diff,HEMOGLOBIN_interval_f1,HEMOGLOBIN_interval_f2,HEMOGLOBIN_diff,"LACTATE, ABG_interval_f1","LACTATE, ABG_interval_f2","LACTATE, ABG_diff",MAGNESIUM_interval_f1,MAGNESIUM_interval_f2,MAGNESIUM_diff,"OXYGEN PERCENT, ABG_interval_f1","OXYGEN PERCENT, ABG_interval_f2","OXYGEN PERCENT, ABG_diff","PCO2, ABG_interval_f1","PCO2, ABG_interval_f2","PCO2, ABG_diff","PH, ABG_interval_f1","PH, ABG_interval_f2","PH, ABG_diff","PHOSPHORUS, INORGANIC_interval_f1","PHOSPHORUS, INORGANIC_interval_f2","PHOSPHORUS, INORGANIC_diff",PLATELET COUNT_interval_f1,PLATELET COUNT_interval_f2,PLATELET COUNT_diff,"PO2, ABG_interval_f1","PO2, ABG_interval_f2","PO2, ABG_diff",POTASSIUM_interval_f1,POTASSIUM_interval_f2,POTASSIUM_diff,SODIUM_interval_f1,SODIUM_interval_f2,SODIUM_diff,WBC_interval_f1,WBC_interval_f2,WBC_diff,HR_mean,O2Sat_mean,SBP_mean,MAP_mean,Resp_mean,O2Sat_median,SBP_median,MAP_median,Resp_median,O2Sat_min,SBP_min,MAP_min,Resp_min,O2Sat_max,SBP_max,MAP_max,Resp_max,O2Sat_std,SBP_std,MAP_std,Resp_std,O2Sat_dstd,SBP_dstd,MAP_dstd,Resp_dstd,HR_score,Temp_score,Resp_score,MAP_score,qsofa,SIRS,PredictedProbability,SepsisLabel_0.005,SepsisLabel_0.01,SepsisLabel_0.02,SepsisLabel_0.03,SepsisLabel_0.04,SepsisLabel_0.05,SepsisLabel_0.06,SepsisLabel_0.07,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time,DischargeInstant,DischargeDateKey,Sepsis_ICD_List,Sepsis_Case,Lactate_Values,Lactate_Ordered,Abx_Medication_Values,Abx_Ordered,Vent_Values,IMV,Vasopressor_Medication_Values,Vasopressor_Ordered,Blood_Culture_Values,Blood_Culture_Ordered,INR_MELD_ResultInstant,INR_MELD_Value,Sepsis_Grouper,Liver_Comorbidity,Cardiovascular_Comorbidity,Respiratory_Comorbidity,Renal_Comorbidity,Immunocompromised_Comorbidity,AKI_Comorbidity,apache_ii_score,apache_ii_Positive,Bilirubin_MELD,Creatinine_MELD,Sodium_MELD,INR_MELD,MELD_Score,MELD_Score_Positive,time_to_first_ML_alert,SI

# Select columns that will be included in the final table

### Laboratory Values

https://www.uptodate.com/contents/sepsis-syndromes-in-adults-epidemiology-definitions-clinical-presentation-diagnosis-and-prognosis?search=sepsis&topicRef=1613&source=see_link

Laboratory signs — Similarly, laboratory features are nonspecific and may be associated with abnormalities due to the underlying cause of sepsis or to tissue hypoperfusion or organ dysfunction from sepsis. They include the following:

●Leukocytosis (white blood cell [WBC] count >12,000 microL–1) or leukopenia (WBC count <4000 microL–1).

●Normal WBC count with greater than 10 percent immature forms.

●Hyperglycemia (plasma glucose >140 mg/dL or 7.7 mmol/L) in the absence of diabetes.

●Plasma C-reactive protein more than two standard deviations above the normal value.

●Arterial hypoxemia (arterial oxygen tension [PaO2]/fraction of inspired oxygen [FiO2] <300).

●Acute oliguria (urine output <0.5 mL/kg/hour for at least two hours despite adequate fluid resuscitation).

●Creatinine increase >0.5 mg/dL or 44.2 micromol/L.

●Coagulation abnormalities (international normalized ratio [INR] >1.5 or activated partial thromboplastin time [aPTT] >60 seconds).

●Thrombocytopenia (platelet count <100,000 microL–1).

●Hyperbilirubinemia (plasma total bilirubin >4 mg/dL or 70 micromol/L).

●Adrenal insufficiency (eg, hyponatremia, hyperkalemia), and the euthyroid sick syndrome can also be found in sepsis.

●Hyperlactatemia (higher than the laboratory upper limit of normal) – An elevated serum lactate (eg, >2 mmol/L) can be a manifestation of organ hypoperfusion in the presence or absence of hypotension and is an important component of the initial evaluation, since elevated lactate is associated with poor prognosis [50,74-76]. A serum lactate level ≥4 mmol/L is consistent with, but not diagnostic of, septic shock. Additional laboratory studies that help characterize the severity of sepsis include a low platelet count, and elevated international normalized ratio, creatinine, and bilirubin. Although arterial and venous lactate correlate, arterial lactate measurements are more accurate and preferred [77].

●Plasma procalcitonin more than two standard deviations above the normal value (not routinely performed in many centers) – Elevated serum procalcitonin levels are associated with bacterial infection and sepsis [78-80]. Despite this, a meta-analysis of 18 studies found that procalcitonin did not readily distinguish sepsis from nonseptic systemic inflammation (sensitivity of 71 percent and specificity of 71 percent) [79]. (See "Evaluation and management of suspected sepsis and septic shock in adults", section on 'De-escalation and duration of antibiotics'.)

●Mid-regional pro-Adrenomedullin (MR-proADM) has been used to predict occurrence and worsening of organ failure in critically ill patients. Yet controversial, monitoring of MR-proADM levels may improve the diagnosis of bacterial infection and contribute to prognosis and antibiotic therapy effectiveness [81].

In [5]:
most_recent_labs = ['WBC', 'GLUCOSE', 'PO2, ABG', 'FIO2, ABG', 'CREATININE', 'APTT', 
                'PLATELET COUNT', 'BILIRUBIN TOTAL', 'LACTATE, ABG']

#baseline_labs = [labs + '_' for labs in include_labs]

include_labs = most_recent_labs

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Vitals

From https://www.uptodate.com/contents/sepsis-syndromes-in-adults-epidemiology-definitions-clinical-presentation-diagnosis-and-prognosis?search=sepsis&topicRef=1613&source=see_link

Symptoms and signs — The symptoms and signs of sepsis are nonspecific but may include the following:

●Symptoms and signs specific to an infectious source (eg, cough and dyspnea may suggest pneumonia, pain and purulent exudate in a surgical wound may suggest an underlying abscess).

●Arterial hypotension (eg, systolic blood pressure [SBP] <90 mmHg, mean arterial pressure [MAP] <70 mmHg, an SBP decrease >40 mmHg, or less than two standard deviations below normal for age). Because a sphygmomanometer may be unreliable in hypotensive patients, an arterial catheter may be needed. (See "Intra-arterial catheterization for invasive monitoring: Indications, insertion techniques, and interpretation".)

●Temperature >38.3 or <36ºC.

●Heart rate >90 beats/min or more than two standard deviations above the normal value for age.

●Tachypnea, respiratory rate >20 breaths/minute.

●Signs of end-organ perfusion:

•Warm, flushed skin may be present in the early phases of sepsis. As sepsis progresses to shock, the skin may become cool due to redirection of blood flow to core organs. Decreased capillary refill, cyanosis, or mottling may indicate shock.

•Additional signs of hypoperfusion include altered mental status, obtundation or restlessness, and oliguria or anuria.

•Ileus or absent bowel sounds are often an end-stage sign of hypoperfusion.

In [6]:
most_recent_vitals = ['SBP', 'MAP', 'Temp', 'HR', 'Resp']

#baseline_vitals = [vital + '_' for vital in most_recent_vitals] 

include_vitals =  most_recent_vitals

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Intervention Columns

In [7]:
include_interventions = ['Sepsis_Case', 'Blood_Culture_Ordered', 'Lactate_Ordered', 
                         'Abx_Ordered', 'Vasopressor_Ordered', 'IMV']

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Sepsis-Related Scoring

In [8]:
include_scores = ['time_to_first_ML_alert','Sepsis3', 'time_to_first_sepsis3_alert', 
                  'SIRS_Positive', 'time_to_first_SIRS_alert', 
                  'qSOFA_Positive','time_to_first_qsofa_alert', 'MEWS_Positive', 'time_to_first_mews_alert']

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Basic demographics

In [9]:
include_demogs = ['AgeAtAdmissioninYears', 'sex_Male']

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Comorbidities

In [10]:
pred_df['Renal_Comorbidity'] = pred_df[['Renal_Comorbidity', 'AKI_Comorbidity']].max(axis=1)

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
pred_df['MELD_Score_Positive'] = pred_df['MELD_Score'].apply(lambda x: 1 if x >= 25 else 0)
pred_df

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,ID,EncounterEpicCsn,PrimaryMRN,DepartmentServiceGrouper,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,HR,Resp,Temp,Weight,BMI,SpO2,gcs_total_score,MAP,ALKALINE PHOSPHATASE,APTT,"ARTERIAL PCO2, POC","ARTERIAL PH, POC","ARTERIAL PO2, POC",AST,BILIRUBIN TOTAL,BLOOD UREA NITROGEN,CALCIUM,CHLORIDE,CREATININE,FIBRINOGEN,"FIO2, ABG",GLUCOSE,HEMATOCRIT,HEMOGLOBIN,"LACTATE, ABG",MAGNESIUM,"OXYGEN PERCENT, ABG","PCO2, ABG","PH, ABG","PHOSPHORUS, INORGANIC",PLATELET COUNT,"PO2, ABG",POTASSIUM,SODIUM,WBC,AgeAtAdmissioninYears,sex_Female,sex_Male,etco2_interval_f1,etco2_interval_f2,etco2_diff,SBP_interval_f1,SBP_interval_f2,SBP_diff,DBP_interval_f1,DBP_interval_f2,DBP_diff,HR_interval_f1,HR_interval_f2,HR_diff,Resp_interval_f1,Resp_interval_f2,Resp_diff,Temp_interval_f1,Temp_interval_f2,Temp_diff,Weight_interval_f1,Weight_interval_f2,Weight_diff,BMI_interval_f1,BMI_interval_f2,BMI_diff,SpO2_interval_f1,SpO2_interval_f2,SpO2_diff,gcs_total_score_interval_f1,gcs_total_score_interval_f2,gcs_total_score_diff,MAP_interval_f1,MAP_interval_f2,MAP_diff,ALKALINE PHOSPHATASE_interval_f1,ALKALINE PHOSPHATASE_interval_f2,ALKALINE PHOSPHATASE_diff,APTT_interval_f1,APTT_interval_f2,APTT_diff,"ARTERIAL PCO2, POC_interval_f1","ARTERIAL PCO2, POC_interval_f2","ARTERIAL PCO2, POC_diff","ARTERIAL PH, POC_interval_f1","ARTERIAL PH, POC_interval_f2","ARTERIAL PH, POC_diff","ARTERIAL PO2, POC_interval_f1","ARTERIAL PO2, POC_interval_f2","ARTERIAL PO2, POC_diff",AST_interval_f1,AST_interval_f2,AST_diff,BILIRUBIN TOTAL_interval_f1,BILIRUBIN TOTAL_interval_f2,BILIRUBIN TOTAL_diff,BLOOD UREA NITROGEN_interval_f1,BLOOD UREA NITROGEN_interval_f2,BLOOD UREA NITROGEN_diff,CALCIUM_interval_f1,CALCIUM_interval_f2,CALCIUM_diff,CHLORIDE_interval_f1,CHLORIDE_interval_f2,CHLORIDE_diff,CREATININE_interval_f1,CREATININE_interval_f2,CREATININE_diff,"FIO2, ABG_interval_f1","FIO2, ABG_interval_f2","FIO2, ABG_diff",GLUCOSE_interval_f1,GLUCOSE_interval_f2,GLUCOSE_diff,HEMATOCRIT_interval_f1,HEMATOCRIT_interval_f2,HEMATOCRIT_diff,HEMOGLOBIN_interval_f1,HEMOGLOBIN_interval_f2,HEMOGLOBIN_diff,"LACTATE, ABG_interval_f1","LACTATE, ABG_interval_f2","LACTATE, ABG_diff",MAGNESIUM_interval_f1,MAGNESIUM_interval_f2,MAGNESIUM_diff,"OXYGEN PERCENT, ABG_interval_f1","OXYGEN PERCENT, ABG_interval_f2","OXYGEN PERCENT, ABG_diff","PCO2, ABG_interval_f1","PCO2, ABG_interval_f2","PCO2, ABG_diff","PH, ABG_interval_f1","PH, ABG_interval_f2","PH, ABG_diff","PHOSPHORUS, INORGANIC_interval_f1","PHOSPHORUS, INORGANIC_interval_f2","PHOSPHORUS, INORGANIC_diff",PLATELET COUNT_interval_f1,PLATELET COUNT_interval_f2,PLATELET COUNT_diff,"PO2, ABG_interval_f1","PO2, ABG_interval_f2","PO2, ABG_diff",POTASSIUM_interval_f1,POTASSIUM_interval_f2,POTASSIUM_diff,SODIUM_interval_f1,SODIUM_interval_f2,SODIUM_diff,WBC_interval_f1,WBC_interval_f2,WBC_diff,HR_mean,O2Sat_mean,SBP_mean,MAP_mean,Resp_mean,O2Sat_median,SBP_median,MAP_median,Resp_median,O2Sat_min,SBP_min,MAP_min,Resp_min,O2Sat_max,SBP_max,MAP_max,Resp_max,O2Sat_std,SBP_std,MAP_std,Resp_std,O2Sat_dstd,SBP_dstd,MAP_dstd,Resp_dstd,HR_score,Temp_score,Resp_score,MAP_score,qsofa,SIRS,PredictedProbability,SepsisLabel_0.005,SepsisLabel_0.01,SepsisLabel_0.02,SepsisLabel_0.03,SepsisLabel_0.04,SepsisLabel_0.05,SepsisLabel_0.06,SepsisLabel_0.07,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time,DischargeInstant,DischargeDateKey,Sepsis_ICD_List,Sepsis_Case,Lactate_Values,Lactate_Ordered,Abx_Medication_Values,Abx_Ordered,Vent_Values,IMV,Vasopressor_Medication_Values,Vasopressor_Ordered,Blood_Culture_Values,Blood_Culture_Ordered,INR_MELD_ResultInstant,INR_MELD_Value,Sepsis_Grouper,Liver_Comorbidity,Cardiovascular_Comorbidity,Respiratory_Comorbidity,Renal_Comorbidity,Immunocompromised_Comorbidity,AKI_Comorbidity,apache_ii_score,apache_ii_Positive,Bilirubin_MELD,Creatinine_MELD,Sodium_MELD,INR_MELD,MELD_Score,MELD_Sco

In [12]:
include_comorbidities = ['MELD_Score', 'MELD_Score_Positive', 'apache_ii_score', 'Liver_Comorbidity',
 'Cardiovascular_Comorbidity', 'Respiratory_Comorbidity','Renal_Comorbidity', 'Immunocompromised_Comorbidity']

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Create Table One

In [13]:
final_cols = include_demogs + include_comorbidities + include_vitals + include_labs + include_scores + include_interventions + ['Sepsis_Grouper']
final_cols

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['AgeAtAdmissioninYears',
 'sex_Male',
 'MELD_Score',
 'MELD_Score_Positive',
 'apache_ii_score',
 'Liver_Comorbidity',
 'Cardiovascular_Comorbidity',
 'Respiratory_Comorbidity',
 'Renal_Comorbidity',
 'Immunocompromised_Comorbidity',
 'SBP',
 'MAP',
 'Temp',
 'HR',
 'Resp',
 'WBC',
 'GLUCOSE',
 'PO2, ABG',
 'FIO2, ABG',
 'CREATININE',
 'APTT',
 'PLATELET COUNT',
 'BILIRUBIN TOTAL',
 'LACTATE, ABG',
 'time_to_first_ML_alert',
 'Sepsis3',
 'time_to_first_sepsis3_alert',
 'SIRS_Positive',
 'time_to_first_SIRS_alert',
 'qSOFA_Positive',
 'time_to_first_qsofa_alert',
 'MEWS_Positive',
 'time_to_first_mews_alert',
 'Sepsis_Case',
 'Blood_Culture_Ordered',
 'Lactate_Ordered',
 'Abx_Ordered',
 'Vasopressor_Ordered',
 'IMV',
 'Sepsis_Grouper']

In [14]:
cat_columns = ['sex_Male', 'MELD_Score_Positive','Liver_Comorbidity','Cardiovascular_Comorbidity',
               'Respiratory_Comorbidity', 'Renal_Comorbidity','Immunocompromised_Comorbidity',  
               'Sepsis_Case', 'Blood_Culture_Ordered',
                'Lactate_Ordered', 'Abx_Ordered', 'Vasopressor_Ordered', 'IMV', 'Sepsis3','SIRS_Positive', 
               'qSOFA_Positive', 'MEWS_Positive']

cat_limit_dict = {col: 1 for col in cat_columns if col != 'MELD_Score_Positive'}
cat_order_dict = {col: ['1.0'] for col in cat_columns if col != 'MELD_Score_Positive'}

cont_columns = [cols for cols in final_cols if cols not in cat_columns and cols != 'Sepsis_Grouper']

rename_dict = {'AgeAtAdmissioninYears': 'Age (Years)', 'sex_Male':'Male sex, n(%)','MELD_Score':'MELD',
               'MELD_Score_Positive':'MELD_Score', 'apache_ii_score':'APACHE II', 
               'Liver_Comorbidity': 'Liver', 'Cardiovascular_Comorbidity':'Cardiovascular', 
               'Respiratory_Comorbidity':'Respiratory', 'Renal_Comorbidity':'Renal',
               'Immunocompromised_Comorbidity':'Immunocompromised', 'SBP': 'SBP, mmHg',
               'MAP':'MAP, mmHg', 'Temp': 'Temperature (Celcius)', 'HR':'Heart rate, beats/min', 
              'Resp':'Respiratory rate, breaths/min', 'WBC':'WBC, 10^9/L', 'GLUCOSE':'Glucose, mg/dL',
              'PO2, ABG':'PO2, mmHg', 'FIO2, ABG': 'FiO2, %', 'CREATININE':'Creatinine, mg/dL',
              'APTT':'aPTT, seconds', 'PLATELET COUNT': 'Platelet Count, 10^9/L', 
               'BILIRUBIN TOTAL':'Bilirubin Total, mg/dL ', 'LACTATE, ABG': 'Lactate, mmol/L', 
               'time_to_first_ML_alert': 'LOS, first ML Alert, hrs', 'Sepsis3':'Sepsis-3 Definition, n(%)', 
               'time_to_first_sepsis3_alert':'LOS, Sepsis-3 Definition','SIRS_Positive':'SIRS >=2, n(%)',
               'time_to_first_SIRS_alert': 'LOS, first SIRS Alert, hrs', 'qSOFA_Positive':'qSOFA >=2, n(%)',
               'time_to_first_qsofa_alert': 'LOS, first qSOFA Alert, hrs','MEWS_Positive':'MEWS >=5, n(%)',
               'time_to_first_mews_alert': 'LOS, first MEWS Alert, hrs', 'Sepsis_Case':'Sepsis ICD Assigned, n(%)',
               'Blood_Culture_Ordered': 'Blood Culture Order, n(%)', 'Lactate_Ordered': 'Lactate Lab Order, n(%)',
               'Abx_Ordered':'Antibiotic Administration, n(%)', 'Vasopressor_Ordered':'Vasopressor and Inotrope Administration, n(%)',
               'IMV':'Mechanical Ventilation, n(%)'
              }

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/asyncio/events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


In [15]:
from tableone import TableOne
df_tableone = TableOne(pred_df, limit=cat_limit_dict, order=cat_order_dict, columns=final_cols, categorical=cat_columns, groupby=['Sepsis_Grouper'], rename=rename_dict, nonnormal=cont_columns, pval=True, label_suffix=False, missing=False, display_all=True)
df_tableone
#df_tableone.to_html('data/tableone_20220805_20220930_20221010.html')

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/gpfs/home/bvg228/.local/lib/python3.8/site-packages/tableone/tableone.py:1217: UserWarning: No P-Value was computed for FIO2, ABG due to the low number of observations.
  warnings.warn(msg)
/gpfs/home/bvg228/.local/lib/python3.8/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/gpfs/home/bvg228/.local/lib/python3.8/site-packages/tableone/tableo

Grouped by Sepsis_Grouper                                                                                                             
                                                                    Overall High Risk, ML Sepsis Score (0.5, 1.0] Medium Risk, ML Sepsis Score [0.25, 0.31)               Others P-Value
n                                                                     13816                                  1834                                      1844                10138        
Age (Years)                                                61.3 [38.3,75.3]                      70.9 [59.3,81.1]                          69.1 [55.8,80.0]     56.1 [35.2,73.0]  <0.001
Male sex, n(%)                                1.0               5611 (40.6)                            999 (54.5)                                925 (50.2)          3687 (36.4)  <0.001
MELD                                                        10.0 [7.0,17.0]                       13.0 [9.0,21.0]                           11.0 [8.0,18.0]       8.0 [7.0,12.0]  <0.001
MELD_Score                                    0                13467 (97.5)                           1649 (89.9)                               1761 (95.5)         10057 (99.2)  <0.001
                                              1                   349 (2.5)                            185 (10.1)                                  83 (4.5)             81 (0.8)        
APACHE II                                                  15.0 [11.0,22.8]                      16.5 [14.0,25.2]                          15.5 [10.8,18.8]      10.0 [6.8,10.8]  <0.001
Liver                                         1.0               2658 (19.2)                            541 (29.5)                                477 (25.9)          1640 (16.2)  <0.001
Cardiovascular                                1.0               2426 (17.6)                            645 (35.2)                                545 (29.6)          1236 (12.2)  <0.001
Respiratory                                   1.0               4743 (34.3)                            980 (53.4)                                870 (47.2)          2893 (28.5)  <0.001
Renal                                         1.0               3415 (24.7)                            914 (49.8)                                742 (40.2)          1759 (17.4)  <0.001
Immunocompromised                             1.0               2204 (16.0)                            518 (28.2)                                437 (23.7)          1249 (12.3)  <0.001
SBP, mmHg                                               127.0 [113.0,144.0]                    106.0 [96.0,119.0]                       113.0 [102.0,128.5]  132.0 [119.0,148.0]  <0.001
MAP, mmHg                                                 92.3 [81.7,103.3]                      76.7 [70.7,85.7]                          80.0 [73.1,91.3]    96.0 [87.7,105.7]  <0.001
Temperature (Celcius)                                      36.8 [36.6,37.0]                      36.9 [36.6,37.6]                          36.8 [36.6,37.3]     36.7 [36.6,36.9]  <0.001
Heart rate, beats/min                                      84.0 [73.0,96.0]                     95.0 [81.0,108.0]                         90.0 [75.5,101.0]     82.0 [71.0,93.0]  <0.001
Respiratory rate, breaths/min                              18.0 [17.0,20.0]                      20.0 [18.0,23.0]                          19.0 [18.0,21.5]     18.0 [17.0,19.0]  <0.001
WBC, 10^9/L                                                 10.6 [7.1,15.3]                       12.6 [8.0,17.2]                            9.5 [6.6,13.8]       8.8 [7.1,11.4]  <0.001
Glucose, mg/dL                                           123.0 [99.0,162.0]                   128.0 [101.0,168.0]                        120.0 [99.0,155.1]    99.5 [90.8,110.0]  <0.001
PO2, mmHg                                                110.0 [84.0,163.6]                    112.5 [84.8,168.0]                        104.0 [83.5,150.0]    89.5 [

In [16]:
print(df_tableone.tabulate(tablefmt = "latex"))

\begin{tabular}{lllllll}
\hline
                                               &     & Overall             & High Risk, ML Sepsis Score (0.5, 1.0]   & Medium Risk, ML Sepsis Score [0.25, 0.31)   & Others              & P-Value   \\
\hline
 n                                             &     & 13816               & 1834                                    & 1844                                        & 10138               &           \\
 Age (Years)                                   &     & 61.3 [38.3,75.3]    & 70.9 [59.3,81.1]                        & 69.1 [55.8,80.0]                            & 56.1 [35.2,73.0]    & \ensuremath{<}0.001    \\
 Male sex, n(\%)                                & 1.0 & 5611 (40.6)         & 999 (54.5)                              & 925 (50.2)                                  & 3687 (36.4)         & \ensuremath{<}0.001    \\
 MELD                                          &     & 10.0 [7.0,17.0]     & 13.0 [9.0,21.0]                         & 11.0 [8.0,18.0]

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
